## Лера Бунтякова, 192

In [1]:
import requests
import re
import bs4
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import csv
from collections import Counter
import pandas as pd

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

## Сбор данных

Я решила использовать сайт с ревью на фильмы - letterboxd.com - и взять с него 90 страниц (80 для составления словаря и 10 для тестов) с отзывами критика Дэвида Эрлиха. 

In [3]:
with open('letterboxd.csv', 'w') as f:
    header = ['movie', 'stars', 'review']
    writer = csv.writer(f)
    writer.writerow(header)

In [4]:
def do_nothing():
    return

In [5]:
def get_page(url):
    r = requests.get(url)
    r.encoding = 'UTF-8'
    return r.text

In [6]:
def get_all_reviews_from_page(url):
    page = get_page(url)
    soup = BeautifulSoup(page)
    page_reviews = soup.find_all('div', {'class':'film-detail-content'})
    return page_reviews

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
def good_text(text):
    tokens = word_tokenize(text)
    good = []
    for token in tokens:
        if not token in '.,?!’/<>$%&"“”\'-)(_…:;.':
            good.append(lemmatizer.lemmatize(token, get_wordnet_pos(token)))
    
    return ' '.join(good) 

In [9]:
def write_reviews_from_page(page_reviews, writer):
    for review in page_reviews:

        movie = review.a.string
        try:
            stars = review.span['class'][2].split('-')[1]
        except IndexError:
            stars = None
        paragraphs = review.find('div', {'class':'body-text -prose collapsible-text'}).find_all('p')
        text = ''

        if paragraphs:
            for paragraph in paragraphs:
                try:
                    text = text + paragraph.string + ' '
                except TypeError:
                    do_nothing()
        

        text = good_text(text.lower())
        
        
        row = [movie, stars, text]
        writer.writerow(row)


In [10]:
def a():
    with open('letterboxd.csv', 'a') as f:
        writer = csv.writer(f)
        for i in range(1, 80):
            url = 'http://letterboxd.com/davidehrlich/films/reviews/page/'+str(i)+'/'
            page_reviews = get_all_reviews_from_page(url)
            write_reviews_from_page(page_reviews, writer)

In [11]:
a()

## Создание списков слов

In [12]:
df = pd.read_csv('letterboxd.csv')

"Хорошими" отзывами будут считаться те, где 8 и больше звезд, "плохими" - те, где 4 и меньше. Использовать 1 и 10 звезд плохо, потому что тогда будет слишком мало данных, а так получается примерно одинаково много (182 и 213). 

In [13]:
df = df.dropna(how="any")
good = df[df['stars']>7]
bad = df[df['stars']<5]

Соберем слова из каждого вида отзывов.

In [14]:
good_words = []
for line in good['review']:
    good_words.extend(line.split())

In [15]:
bad_words = []
for line in bad['review']:
    bad_words.extend(line.split())

И найдем уникальные.

In [16]:
set_bad_words = set(bad_words)
set_good_words = set(good_words)

In [17]:
very_good_words = set_good_words - set_bad_words
very_bad_words = set_bad_words - set_good_words

Второй список почти в полтора раза длиннее первого, что приведет к проблемам: при определении тональности один вариант будет перевешивать.

In [18]:
len(very_good_words)

2023

In [19]:
len(very_bad_words)

2898

Поэтому возьмем из каждого списка 2000 самых популярных слов и найдем уникальные среди них. Эти списки и будем использовать.

In [20]:
popular_good_words = set(w[0] for w in Counter(good_words).most_common(2000))

In [21]:
popular_bad_words = set(w[0] for w in Counter(bad_words).most_common(2000))

In [22]:
popular_very_good_words = popular_good_words - popular_bad_words
popular_very_bad_words = popular_bad_words - popular_good_words

In [23]:
len(popular_very_bad_words)

1180

In [24]:
len(popular_very_good_words)

1180

In [25]:
def decide_if_good(review, very_good_words, very_bad_words):
    review = good_text(review)
    is_good = 0
    is_bad = 0
    for word in review.split():
        if word in very_good_words:
            is_good += 1
        if word in very_bad_words:
            is_bad += 1
    if is_good > is_bad:
        return 'good'
    elif is_bad > is_good:
        return 'bad'
    else:
        return '?'

## Тесты

Сначала дособерем дополнительные 10 страниц ревью.

In [26]:
with open('letterboxd_test.csv', 'w') as f:
    header = ['movie', 'stars', 'review']
    writer = csv.writer(f)
    writer.writerow(header)

In [27]:
def a_test():
    with open('letterboxd_test.csv', 'a') as f:
        writer = csv.writer(f)
        for i in range(80, 90):
            url = 'http://letterboxd.com/davidehrlich/films/reviews/page/'+str(i)+'/'
            page_reviews = get_all_reviews_from_page(url)
            write_reviews_from_page(page_reviews, writer)

In [28]:
a_test()

In [29]:
df_test = pd.read_csv('letterboxd_test.csv')
df_test = df_test.dropna(how='any')

Запустим тесты и посчитаем accuracy.

In [30]:
success = 0
fail = 0

for index, row in df_test.iterrows():
    movie = row['movie']
    stars = row['stars']
    review = row['review']
    if stars > 7:
        true_rating = 'good'
    elif stars < 5:
        true_rating = 'bad'
    else:
        true_rating = 'middle'
        continue
        
    my_rating = decide_if_good(review, popular_very_good_words, popular_very_bad_words)
    if my_rating == true_rating:
        success +=1
    else:
        fail += 1
        
print('Accuracy: ', success/(success+fail))

Accuracy:  0.5892857142857143


Предложения по улучшению: 
   1. Использовать не слова, а коллокации, чтобы разделить штуки типа 'was horrible' и 'was not horrible'
   2. Посмотреть на длину хороших и плохих отзывов, на длину предложений в них, на использование смайликов и другие похожие вещи.
   3. Сравнить количество восклицательных знаков